## 1. Importação e Configuração Base

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from typing import Tuple, List
import random

# Configuração para reprodutibilidade
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Configuração do ambiente (do Exercício 1)
GRID_SIZE = 10
NUM_STATES = GRID_SIZE * GRID_SIZE
ESTADO_INICIAL = 1
ESTADO_OBJETIVO = 100

PAREDES = {
    23, 24, 25, 26,
    33, 34, 35, 36,
    54, 64, 74, 84,
    55, 65, 75, 85,
}

ACOES = ['UP', 'DOWN', 'LEFT', 'RIGHT']
MAX_PASSOS_POR_EPISODIO = 1000

print("Configuração base importada com sucesso!")

## 2. Funções do Ambiente (do Exercício 1)

In [ ]:
def transicao_estado(estado: int, acao: str) -> int:
    """Calcula o próximo estado após aplicar uma ação."""
    linha = (estado - 1) // GRID_SIZE
    coluna = (estado - 1) % GRID_SIZE
    
    nova_linha, nova_coluna = linha, coluna
    
    if acao == 'UP':
        nova_linha = linha - 1
    elif acao == 'DOWN':
        nova_linha = linha + 1
    elif acao == 'LEFT':
        nova_coluna = coluna - 1
    elif acao == 'RIGHT':
        nova_coluna = coluna + 1
    
    if nova_linha < 0 or nova_linha >= GRID_SIZE or nova_coluna < 0 or nova_coluna >= GRID_SIZE:
        return estado
    
    novo_estado = nova_linha * GRID_SIZE + nova_coluna + 1
    if novo_estado in PAREDES:
        return estado
    
    return novo_estado


def recompensa(estado: int) -> float:
    """Retorna a recompensa de um estado."""
    return 100.0 if estado == ESTADO_OBJETIVO else 0.0


def acao_aleatoria() -> str:
    """Escolhe uma ação aleatória."""
    return np.random.choice(ACOES)


print("Funções do ambiente definidas!")

## 3. Configuração do Q-Learning

In [ ]:
# Parâmetros do Q-Learning
ALPHA = 0.7  # Taxa de aprendizagem
GAMMA = 0.99  # Fator de desconto

# Parâmetros de treino
NUM_PASSOS_TREINO = 20000
NUM_PASSOS_TESTE = 1000
NUM_EXPERIENCIAS = 30

# Pontos de teste durante o treino
PONTOS_TESTE = [100, 200, 500, 600, 700, 800, 900, 1000, 
                2500, 5000, 7500, 10000, 12500, 15000, 17500, 20000]

# Mapeamento de ações para índices
ACAO_PARA_INDICE = {acao: i for i, acao in enumerate(ACOES)}
INDICE_PARA_ACAO = {i: acao for i, acao in enumerate(ACOES)}

print("Configuração do Q-Learning:")
print(f"  α (alpha) = {ALPHA}")
print(f"  γ (gamma) = {GAMMA}")
print(f"  Número de passos de treino: {NUM_PASSOS_TREINO}")
print(f"  Número de experiências: {NUM_EXPERIENCIAS}")

## 4. Funções Auxiliares de Q-Learning

In [ ]:
def inicializar_Q():
    """Inicializa a tabela Q com zeros."""
    return np.zeros((NUM_STATES + 1, len(ACOES)))


def atualizar_Q(Q, estado, acao, proximo_estado, recompensa_recebida):
    """Atualiza a tabela Q usando a equação de Q-Learning."""
    indice_acao = ACAO_PARA_INDICE[acao]
    q_atual = Q[estado, indice_acao]
    max_q_proximo = np.max(Q[proximo_estado, :])
    novo_q = (1 - ALPHA) * q_atual + ALPHA * (recompensa_recebida + GAMMA * max_q_proximo)
    Q[estado, indice_acao] = novo_q


def escolher_melhor_acao(Q, estado):
    """Escolhe a melhor ação com desempate aleatório."""
    valores_q = Q[estado, :]
    max_q = np.max(valores_q)
    acoes_maximas = [i for i, q in enumerate(valores_q) if q == max_q]
    indice_escolhido = np.random.choice(acoes_maximas)
    return INDICE_PARA_ACAO[indice_escolhido]


def testar_politica_Q(Q, num_passos=NUM_PASSOS_TESTE):
    """Testa a política aprendida sem alterar Q."""
    estado_atual = ESTADO_INICIAL
    recompensa_total = 0.0
    passos_executados = 0
    
    for _ in range(num_passos):
        acao = escolher_melhor_acao(Q, estado_atual)
        proximo_estado = transicao_estado(estado_atual, acao)
        r = recompensa(proximo_estado)
        recompensa_total += r
        passos_executados += 1
        
        if proximo_estado == ESTADO_OBJETIVO:
            proximo_estado = ESTADO_INICIAL
        
        estado_atual = proximo_estado
    
    return recompensa_total / passos_executados if passos_executados > 0 else 0.0


print("Funções de Q-Learning implementadas!")

## 5. Exercício 2.a) Treino com Random Walk

In [ ]:
def treino_random_walk(seed=None):
    """Executa treino com Random Walk."""
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)
    
    inicio = time.time()
    Q = inicializar_Q()
    estado_atual = ESTADO_INICIAL
    recompensas_teste = []
    
    for passo in range(1, NUM_PASSOS_TREINO + 1):
        acao = acao_aleatoria()
        proximo_estado = transicao_estado(estado_atual, acao)
        r = recompensa(proximo_estado)
        atualizar_Q(Q, estado_atual, acao, proximo_estado, r)
        
        if proximo_estado == ESTADO_OBJETIVO:
            proximo_estado = ESTADO_INICIAL
        
        estado_atual = proximo_estado
        
        if passo in PONTOS_TESTE:
            recomp_teste = testar_politica_Q(Q)
            recompensas_teste.append(recomp_teste)
    
    tempo_total = time.time() - inicio
    return Q, recompensas_teste, tempo_total


print("Função de treino Random Walk definida!")

In [ ]:
print("="*80)
print("TREINO COM RANDOM WALK - 30 EXPERIÊNCIAS")
print("="*80)

todas_recompensas_random = []
todos_tempos_random = []
todas_Q_random = []

print(f"\nExecutando {NUM_EXPERIENCIAS} experiências...\n")

for exp in range(1, NUM_EXPERIENCIAS + 1):
    Q, recompensas, tempo = treino_random_walk(seed=42 + exp)
    todas_recompensas_random.append(recompensas)
    todos_tempos_random.append(tempo)
    todas_Q_random.append(Q.copy())
    
    if exp % 5 == 0:
        print(f"Experiência {exp:2d}: tempo = {tempo:.2f}s, recompensa final = {recompensas[-1]:.6f}")

todas_recompensas_random = np.array(todas_recompensas_random)
todos_tempos_random = np.array(todos_tempos_random)

print(f"\n{'='*80}")
print("RESULTADOS - RANDOM WALK")
print(f"{'='*80}")
print(f"\nTempo médio: {np.mean(todos_tempos_random):.2f}s ± {np.std(todos_tempos_random):.2f}s")
print(f"Recompensa final média: {np.mean(todas_recompensas_random[:, -1]):.6f}")
print(f"{'='*80}")

## 6. Exercício 2.b) Treino Greedy

In [ ]:
def treino_greedy(seed=None):
    """Executa treino Greedy (sempre melhor ação)."""
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)
    
    inicio = time.time()
    Q = inicializar_Q()
    estado_atual = ESTADO_INICIAL
    recompensas_teste = []
    
    for passo in range(1, NUM_PASSOS_TREINO + 1):
        acao = escolher_melhor_acao(Q, estado_atual)
        proximo_estado = transicao_estado(estado_atual, acao)
        r = recompensa(proximo_estado)
        atualizar_Q(Q, estado_atual, acao, proximo_estado, r)
        
        if proximo_estado == ESTADO_OBJETIVO:
            proximo_estado = ESTADO_INICIAL
        
        estado_atual = proximo_estado
        
        if passo in PONTOS_TESTE:
            recomp_teste = testar_politica_Q(Q)
            recompensas_teste.append(recomp_teste)
    
    tempo_total = time.time() - inicio
    return Q, recompensas_teste, tempo_total


print("Função de treino Greedy definida!")

In [ ]:
print("="*80)
print("TREINO GREEDY - 30 EXPERIÊNCIAS")
print("="*80)

todas_recompensas_greedy = []
todos_tempos_greedy = []
todas_Q_greedy = []

print(f"\nExecutando {NUM_EXPERIENCIAS} experiências...\n")

for exp in range(1, NUM_EXPERIENCIAS + 1):
    Q, recompensas, tempo = treino_greedy(seed=42 + exp)
    todas_recompensas_greedy.append(recompensas)
    todos_tempos_greedy.append(tempo)
    todas_Q_greedy.append(Q.copy())
    
    if exp % 5 == 0:
        print(f"Experiência {exp:2d}: tempo = {tempo:.2f}s, recompensa final = {recompensas[-1]:.6f}")

todas_recompensas_greedy = np.array(todas_recompensas_greedy)
todos_tempos_greedy = np.array(todos_tempos_greedy)

print(f"\n{'='*80}")
print("RESULTADOS - GREEDY")
print(f"{'='*80}")
print(f"\nTempo médio: {np.mean(todos_tempos_greedy):.2f}s ± {np.std(todos_tempos_greedy):.2f}s")
print(f"Recompensa final média: {np.mean(todas_recompensas_greedy[:, -1]):.6f}")
print(f"{'='*80}")

## 7. Visualização: Comparação Random Walk vs Greedy

In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))

medias_random = np.mean(todas_recompensas_random, axis=0)
desvios_random = np.std(todas_recompensas_random, axis=0)
medias_greedy = np.mean(todas_recompensas_greedy, axis=0)
desvios_greedy = np.std(todas_recompensas_greedy, axis=0)

ax.plot(PONTOS_TESTE, medias_random, 'b-', linewidth=2.5, marker='o', 
        markersize=7, label='Random Walk', zorder=3)
ax.fill_between(PONTOS_TESTE, medias_random - desvios_random, 
                medias_random + desvios_random, alpha=0.2, color='blue')

ax.plot(PONTOS_TESTE, medias_greedy, 'r-', linewidth=2.5, marker='s', 
        markersize=7, label='Greedy', zorder=3)
ax.fill_between(PONTOS_TESTE, medias_greedy - desvios_greedy, 
                medias_greedy + desvios_greedy, alpha=0.2, color='red')

ax.set_xlabel('Número de Passos de Treino', fontsize=13, fontweight='bold')
ax.set_ylabel('Recompensa Média por Passo (Teste)', fontsize=13, fontweight='bold')
ax.set_title('Comparação: Random Walk vs Greedy - Q-Learning\n'
             f'(α={ALPHA}, γ={GAMMA}, {NUM_EXPERIENCIAS} experiências)', 
             fontsize=14, fontweight='bold', pad=15)
ax.grid(True, alpha=0.3)
ax.legend(fontsize=12, loc='lower right', framealpha=0.9)

plt.tight_layout()
plt.savefig('exercicio2_comparacao_random_greedy.png', dpi=300, bbox_inches='tight')
plt.show()

print("Gráfico salvo como 'exercicio2_comparacao_random_greedy.png'")

## 8. Mapa de Calor da Utilidade Aprendida (Figura 3)

In [ ]:
def criar_mapa_calor_utilidade(Q, titulo="Mapa de Calor da Utilidade Aprendida", 
                               nome_ficheiro="mapa_calor.png"):
    """Cria mapa de calor mostrando U(s) = max_a Q[s,a]."""
    utilidades = np.zeros((GRID_SIZE, GRID_SIZE))
    
    for estado in range(1, NUM_STATES + 1):
        linha = (estado - 1) // GRID_SIZE
        coluna = (estado - 1) % GRID_SIZE
        utilidades[linha, coluna] = np.max(Q[estado, :])
    
    for estado in PAREDES:
        linha = (estado - 1) // GRID_SIZE
        coluna = (estado - 1) % GRID_SIZE
        utilidades[linha, coluna] = np.nan
    
    fig, ax = plt.subplots(figsize=(12, 11))
    im = ax.imshow(utilidades, cmap='YlOrRd', interpolation='nearest')
    
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('Utilidade Máxima U(s) = max Q[s,a]', fontsize=12, fontweight='bold')
    
    for estado in range(1, NUM_STATES + 1):
        linha = (estado - 1) // GRID_SIZE
        coluna = (estado - 1) % GRID_SIZE
        
        if estado in PAREDES:
            ax.text(coluna, linha, 'X', ha='center', va='center',
                   color='black', fontsize=14, fontweight='bold')
        else:
            utilidade = utilidades[linha, coluna]
            cor_texto = 'white' if utilidade > np.nanmax(utilidades) * 0.5 else 'black'
            ax.text(coluna, linha, f'{estado}\n{utilidade:.1f}', 
                   ha='center', va='center', color=cor_texto, 
                   fontsize=8, fontweight='bold')
    
    ax.set_xticks(range(GRID_SIZE))
    ax.set_yticks(range(GRID_SIZE))
    ax.set_xticklabels(range(1, GRID_SIZE + 1))
    ax.set_yticklabels(range(1, GRID_SIZE + 1))
    ax.set_xlabel('Coluna', fontsize=12, fontweight='bold')
    ax.set_ylabel('Linha', fontsize=12, fontweight='bold')
    ax.set_title(titulo, fontsize=14, fontweight='bold', pad=20)
    
    ax.set_xticks(np.arange(-0.5, GRID_SIZE, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, GRID_SIZE, 1), minor=True)
    ax.grid(which='minor', color='gray', linestyle='-', linewidth=1)
    ax.tick_params(which='minor', size=0)
    
    plt.tight_layout()
    plt.savefig(nome_ficheiro, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Mapa de calor salvo como '{nome_ficheiro}'")


# Mapas de calor para ambas as estratégias
print("Criando mapas de calor...\n")

criar_mapa_calor_utilidade(
    todas_Q_random[-1], 
    titulo="Mapa de Calor - Random Walk com Q-Learning",
    nome_ficheiro="exercicio2a_mapa_calor_random_walk.png"
)

criar_mapa_calor_utilidade(
    todas_Q_greedy[-1], 
    titulo="Mapa de Calor - Greedy com Q-Learning",
    nome_ficheiro="exercicio2b_mapa_calor_greedy.png"
)

## 9. Análise Comparativa Final

In [ ]:
print("="*80)
print("ANÁLISE COMPARATIVA: RANDOM WALK vs GREEDY")
print("="*80)

print("\n1. TEMPO DE EXECUÇÃO:")
print(f"   Random Walk: {np.mean(todos_tempos_random):.2f}s ± {np.std(todos_tempos_random):.2f}s")
print(f"   Greedy:      {np.mean(todos_tempos_greedy):.2f}s ± {np.std(todos_tempos_greedy):.2f}s")

recomp_final_random = todas_recompensas_random[:, -1]
recomp_final_greedy = todas_recompensas_greedy[:, -1]

print("\n2. RECOMPENSA FINAL (após 20000 passos):")
print(f"   Random Walk: {np.mean(recomp_final_random):.6f} ± {np.std(recomp_final_random):.6f}")
print(f"   Greedy:      {np.mean(recomp_final_greedy):.6f} ± {np.std(recomp_final_greedy):.6f}")

melhoria = ((np.mean(recomp_final_greedy) - np.mean(recomp_final_random)) / 
            np.mean(recomp_final_random) * 100)
print(f"\n   Melhoria do Greedy: {melhoria:+.1f}%")

print("\n" + "="*80)

## 10. Conclusões do Exercício 2

### Random Walk com Q-Learning

**Vantagens:**
- Exploração ampla do espaço de estados
- Descoberta de múltiplos caminhos
- Menos suscetível a mínimos locais

**Desvantagens:**
- Convergência lenta
- Muitas ações desperdiçadas
- Variabilidade alta

### Greedy com Q-Learning

**Vantagens:**
- Convergência rápida
- Exploração focada
- Resultados consistentes

**Desvantagens:**
- Risco de mínimos locais
- Exploração limitada
- Dependência da inicialização

### Melhor Ação por Estado

A tabela Q final permite identificar a **melhor ação para qualquer estado**:

```python
melhor_acao = argmax_a Q[s, a]
```

### Próximos Passos

O **Exercício 3** implementará a estratégia **ε-greedy**, combinando:
- Exploração (ações aleatórias com probabilidade ε)
- Exploração (melhor ação com probabilidade 1-ε)

Esta abordagem híbrida tende a superar ambas as estratégias puras.